In [1]:
import os
import requests
import json
import textwrap as tw
import pandas as pd
import geovpylib.database as db
import geovdata.sparql as sparql
from gmpykit.ipython import infos


# Connect to S&B database
db_url = os.getenv('YELLOW_SWITZERLAND_AND_BEYOND')
execute = True
db.connect_external(db_url, execute)

# Connect to wikidata
sparql.connect_external("https://query.wikidata.org/sparql")


def ask_ollama(prompt, model='mistral', print_prompt=False):

    if print_prompt:
        print()
        print('===== Prompt sent to', model, '=====')
        print(prompt)
        print('====================================')

    url='http://localhost:11434/api/generate'
    response = requests.post(url, json={'model':model,'prompt':prompt})
    text = response.text.strip()
    lines = text.split('\n')
    tokens = list(map(lambda line: json.loads(line)['response'], lines))
    formated = ''.join(tokens)
    answer = formated.strip()
    return answer

[DB] Connecting to PGSQL Database ... Connected!
>> External SPARQL URL set to <https://query.wikidata.org/sparql>
SPARQL URL: https://query.wikidata.org/sparql


# Reset database

In [2]:
# db.execute("delete from hls.occupation")

## Fetch data

In [3]:
persons = db.query("select * from hls.person")
persons['hls_id'] = persons['url'].str.replace('https://hls-dhs-dss.ch/fr/articles/', '')
infos(persons)

Shape:  (25346, 6) - extract:


,id,url,name,notice,uri_geov,hls_id
0,66,https://hls-dhs-dss.ch/fr/articles/024493,"Sciascia, Domenico","Naît le 1599 ou 1603 à Roveredo (GR), meurt le...",None,024493
1,17,https://hls-dhs-dss.ch/fr/articles/023451,"Courten, Jean-Louis de","Naît le 3.7.1766 à Barcelone, meurt le 3.7.176...",None,023451
2,24,https://hls-dhs-dss.ch/fr/articles/024192,"Reding, Nazar","Naît le 11.3.1759 à Schwytz, meurt le 11.3.175...",None,024192
3,29,https://hls-dhs-dss.ch/fr/articles/025343,"Fraefel, Jakob","Naît le 17.2.1926 à Bâle, cath., d'Uzwil et Ge...",None,025343
4,38,https://hls-dhs-dss.ch/fr/articles/024553,"Rossi, Domenico Egidio","1.9.1659 à Fano (Marches), meurt le 28.12.1657...",None,024553


## From text (with Mistral)

In [4]:
prompt_profession = """
From the following text, I would like to extract all profession of "__subject__" (only the ones clearly mentioned!), in the following format:
- Profession: <eg: Carpenter> | profession start date: <eg: 2020> | profession end date: <eg: 2021> | profession main place: <eg: Paris>

Answer needs to be the list only with no comments, no introduction, no notes and no explanations.


Here is the text:
"__text__"
"""

def from_mistral(name, notice, print_prompt=False): 
    prompt = prompt_profession.replace('__subject__', name).replace('__text__', notice)
    occupations = ask_ollama(prompt, print_prompt=print_prompt)

    print('')
    print('---------------------')
    print('From Mistral:')
    print(occupations)

# From wikidata

In [5]:
def from_wikidata(hls_id):
    occupations = sparql.query('''
        select 
            ?wikidata_url ?occupationLabel ?start_date ?end_date ?location
        where {
            ?wikidata_url wdt:P902 "''' + hls_id + '''" .
            ?wikidata_url p:P106  ?occqual .
            ?occqual      ps:P106 ?occupation .
            optional { ?occqual pq:P580 ?start_date . }   
            optional { ?occqual pq:P582 ?end_date . }
            optional { ?occqual pq:P937 ?location . }
            service wikibase:label { bd:serviceParam wikibase:language "en". } 
        }
    ''')

    # Add missing columns
    needed_columns = ['occupation', 'start_date', 'end_date', 'location']
    for col in needed_columns:
        if col not in occupations.columns:
            occupations[col] = pd.NA

    if len(occupations): wikidata_url = occupations.iloc[0]['wikidata_url']
    else: wikidata_url = "No URL"
    occupations.drop(columns=['wikidata_url'], inplace=True)

    # Rename columns
    occupations.rename(inplace=True, columns={'occupationLabel':'occupation'})

    print('')
    print('---------------------')
    print('From Wikidata (' + wikidata_url + '):')
    display(occupations)

# Tests

In [6]:
person = persons.sample(1).iloc[0]


print('')
print('---------------------')
print(person['name'])
print(tw.fill(person['notice'], width=100))

from_mistral(person['name'], person['notice'])

from_wikidata(person['hls_id'])



---------------------
Herold, Johannes Basilius
Naît le 17.12.1514 à Höchstädt an der Donau, meurt le 17.12.1514 à Bâle, prot. (cath. avant 1539),
de Bâle (reçu bourgeois en 1556). Fils illégitime de Johannes, bailli des Fugger. 1) NN Gernler,
d'une famille bourgeoise de Bâle, 2) Veronika Blowner, fille du conseiller Bernhard. Ecole à
Donauwörth, puis années de voyage, surtout en Italie. A Bâle dès 1539. Sans être théologien, H. fut
pasteur à Reinach (BL) en 1542-1543, candidat au pastorat et diacre à Augsbourg (1543-1544) et
pasteur à Pfeffingen (1545-1552). De retour à Bâle dès 1554, il travailla dans une imprimerie comme
correcteur et traducteur. Jean Oporin, Heinrich Petri et Pietro Perna imprimèrent les textes
antiques et humanistes qu'il édita avec ses propres commentaires. L'édition des Antiquitates
Germanicae (1557), recueil de lois des peuples barbares en ancien haut allemand, est d'une
importance considérable pour l'histoire du droit au haut Moyen Age. Les historiens de la l

,occupation,occupation,start_date,end_date,location
0,printer,<NA>,<NA>,<NA>,<NA>
1,translator,<NA>,<NA>,<NA>,<NA>
2,corrector,<NA>,<NA>,<NA>,<NA>
